# Clustering

# Этап 1. Предобработка.

Подключаем бибилиотеки:

In [1]:
import numpy as np
import pandas as pd

Считываем данные:

In [2]:
data = pd.read_csv('data.csv')
data.head(5)

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,1,0,12/7/2009,ADESA,2006,3,MAZDA,MAZDA3,i,4D SEDAN I,...,11597.0,12409.0,NaN,NaN,21973,33619,FL,7100.0,0,1113
1,2,0,12/7/2009,ADESA,2004,5,DODGE,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,...,11374.0,12791.0,NaN,NaN,19638,33619,FL,7600.0,0,1053
2,3,0,12/7/2009,ADESA,2005,4,DODGE,STRATUS V6,SXT,4D SEDAN SXT FFV,...,7146.0,8702.0,NaN,NaN,19638,33619,FL,4900.0,0,1389
3,4,0,12/7/2009,ADESA,2004,5,DODGE,NEON,SXT,4D SEDAN,...,4375.0,5518.0,NaN,NaN,19638,33619,FL,4100.0,0,630
4,5,0,12/7/2009,ADESA,2005,4,FORD,FOCUS,ZX3,2D COUPE ZX3,...,6739.0,7911.0,NaN,NaN,19638,33619,FL,4000.0,0,1020


Получаем информацию с файла:

In [3]:
f = open('Data_Dictionary.txt', 'r')
while True:
    line = f.readline()
    if line == '':
        break
    print(line, end='')
f.close()

Field Name				Definition
RefID				        Unique (sequential) number assigned to vehicles
IsBadBuy				Identifies if the kicked vehicle was an avoidable purchase 
PurchDate				The Date the vehicle was Purchased at Auction
Auction					Auction provider at which the  vehicle was purchased
VehYear					The manufacturer's year of the vehicle
VehicleAge				The Years elapsed since the manufacturer's year
Make					Vehicle Manufacturer 
Model					Vehicle Model
Trim					Vehicle Trim Level
SubModel				Vehicle Submodel
Color					Vehicle Color
Transmission				Vehicles transmission type (Automatic, Manual)
WheelTypeID				The type id of the vehicle wheel
WheelType				The vehicle wheel type description (Alloy, Covers)
VehOdo					The vehicles odometer reading
Nationality				The Manufacturer's country
Size					The size category of the vehicle (Compact, SUV, etc.)
TopThreeAmericanName			Identifies if the manufacturer is one of the top three American manufacturers
MMRAcquisitionAuctionAveragePrice	Acquisiti

Что имеем на самом деле:

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72983 entries, 0 to 72982
Data columns (total 34 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   RefId                              72983 non-null  int64  
 1   IsBadBuy                           72983 non-null  int64  
 2   PurchDate                          72983 non-null  object 
 3   Auction                            72983 non-null  object 
 4   VehYear                            72983 non-null  int64  
 5   VehicleAge                         72983 non-null  int64  
 6   Make                               72983 non-null  object 
 7   Model                              72983 non-null  object 
 8   Trim                               70623 non-null  object 
 9   SubModel                           72975 non-null  object 
 10  Color                              72975 non-null  object 
 11  Transmission                       72974 non-null  obj

Выделяем избыточные и неинформативные колонки и колонки с категориальными признаками

In [5]:
drop_columns = [
    'RefId', # ид неинформативен
    'PurchDate', # дата тяжело интерпретировать
    'Model', # есть страна производства
    'SubModel', # есть страна производства
    'Trim', # 
    'WheelTypeID', # избыточная информация, есть WhellType
    'Make', # производитель заменен на страну производства
    'VNST', # машину можно перегранать из штата в штат, малоинформативно (есть название аукциона)
    'VNZIP1', # тоже, что и VNST
    'BYRNO', # ид покупателя может дать инфу, но очень косвенно
    'MMRAcquisitionAuctionCleanPrice', # лишняя информация
    'MMRAcquisitionRetailAveragePrice',
    'MMRAcquisitonRetailCleanPrice',
    'MMRCurrentAuctionCleanPrice',
    'MMRCurrentRetailAveragePrice',
    'MMRCurrentRetailCleanPrice' 
]
cat_columns = ['Auction', 'WheelType', 'Color', 'Transmission', 'Nationality', 'Size', 'TopThreeAmericanName', 'PRIMEUNIT', 'AUCGUART']

Исключаем лишние признаки:

In [6]:
data = data.drop(columns=drop_columns)
data.head(5)

,IsBadBuy,Auction,VehYear,VehicleAge,Color,Transmission,WheelType,VehOdo,Nationality,Size,TopThreeAmericanName,MMRAcquisitionAuctionAveragePrice,MMRCurrentAuctionAveragePrice,PRIMEUNIT,AUCGUART,VehBCost,IsOnlineSale,WarrantyCost
0,0,ADESA,2006,3,RED,AUTO,Alloy,89046,OTHER ASIAN,MEDIUM,OTHER,8155.0,7451.0,NaN,NaN,7100.0,0,1113
1,0,ADESA,2004,5,WHITE,AUTO,Alloy,93593,AMERICAN,LARGE TRUCK,CHRYSLER,6854.0,7456.0,NaN,NaN,7600.0,0,1053
2,0,ADESA,2005,4,MAROON,AUTO,Covers,73807,AMERICAN,MEDIUM,CHRYSLER,3202.0,4035.0,NaN,NaN,4900.0,0,1389
3,0,ADESA,2004,5,SILVER,AUTO,Alloy,65617,AMERICAN,COMPACT,CHRYSLER,1893.0,1844.0,NaN,NaN,4100.0,0,630
4,0,ADESA,2005,4,SILVER,MANUAL,Covers,69367,AMERICAN,COMPACT,FORD,3913.0,3247.0,NaN,NaN,4000.0,0,1020


Займемся устранением пустот:

In [7]:
data.isnull().sum()

IsBadBuy                                 0
Auction                                  0
VehYear                                  0
VehicleAge                               0
Color                                    8
Transmission                             9
WheelType                             3174
VehOdo                                   0
Nationality                              5
Size                                     5
TopThreeAmericanName                     5
MMRAcquisitionAuctionAveragePrice       18
MMRCurrentAuctionAveragePrice          315
PRIMEUNIT                            69564
AUCGUART                             69564
VehBCost                                 0
IsOnlineSale                             0
WarrantyCost                             0
dtype: int64

In [8]:
data.loc[data['Color'].isna(), 'Color'] = 'OTHER'
data.loc[data['WheelType'].isna(), 'WheelType'] = 'Special'
data.loc[data['Transmission'] == 'Manual', 'Transmission'] = 'MANUAL'
data.loc[data['Transmission'].isna(), 'Transmission'] = 'AUTO'
data.loc[data['Nationality'].isna(), 'Nationality'] = 'AMERICAN'
data.loc[data['Size'].isna(), 'Size'] = 'MEDIUM'
data.loc[data['TopThreeAmericanName'].isna(), 'TopThreeAmericanName'] = 'OTHER'
data.loc[data['PRIMEUNIT'].isna(), 'PRIMEUNIT'] = 'OTHER'
data.loc[data['AUCGUART'].isna(), 'AUCGUART'] = 'OTHER'
data.loc[data['MMRAcquisitionAuctionAveragePrice'].isna(), 'MMRAcquisitionAuctionAveragePrice'] = data['MMRAcquisitionAuctionAveragePrice'].mean(skipna=True)
data.loc[data['MMRCurrentAuctionAveragePrice'].isna(), 'MMRCurrentAuctionAveragePrice'] = data['MMRCurrentAuctionAveragePrice'].mean(skipna=True)

In [9]:
y = data['IsBadBuy'].to_numpy()
X = data.drop(columns=['IsBadBuy'] + cat_columns).to_numpy()

K-means:

In [10]:
from sklearn.cluster import KMeans

In [11]:
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X_train, X_test, y_train, y_test = X[indices[:65000]], X[indices[65000:]], y[indices[:65000]], y[indices[65000:]]

In [12]:
%%time
model = KMeans(n_clusters=2)
model.fit(X_train)
print('Точность на train:', (model.labels_ == y_train).sum() / len(y_train))
print('Точность на test:', (model.predict(X_test) == y_test).sum() / len(y_test))

Точность на train: 0.5737692307692308
Точность на test: 0.5654515846173118
Wall time: 391 ms


Добавим категориальные признаки:

In [13]:
y = data['IsBadBuy'].to_numpy()
X = pd.get_dummies(data.drop(columns=['IsBadBuy'])).to_numpy()

indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X_train, X_test, y_train, y_test = X[indices[:65000]], X[indices[65000:]], y[indices[:65000]], y[indices[65000:]]

In [14]:
%%time
model = KMeans(n_clusters=2)
model.fit(X_train)
print('Точность на train:', (model.labels_ == y_train).sum() / len(y_train))
print('Точность на test:', (model.predict(X_test) == y_test).sum() / len(y_test))

Точность на train: 0.5733384615384616
Точность на test: 0.5628209946135538
Wall time: 809 ms
